<a href="https://colab.research.google.com/github/enesemretas/hingeprot-colab-ui/blob/main/notebooks/HingeProt_UI.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
# @title
%%bash
cd /content
rm -rf hingeprot-colab-ui
git clone https://github.com/enesemretas/hingeprot-colab-ui.git
cd hingeprot-colab-ui

apt-get update -qq
apt-get install -y -qq perl libblas3 dos2unix

# install libg2c.so.0 (legacy g77 runtime)
wget -q https://old-releases.ubuntu.com/ubuntu/pool/universe/g/gcc-3.4/gcc-3.4-base_3.4.6-6ubuntu3_amd64.deb
wget -q https://old-releases.ubuntu.com/ubuntu/pool/universe/g/gcc-3.4/libg2c0_3.4.6-6ubuntu3_amd64.deb
dpkg -i gcc-3.4-base_3.4.6-6ubuntu3_amd64.deb libg2c0_3.4.6-6ubuntu3_amd64.deb || true
apt-get -y -qq -f install
ldconfig

chmod +x hingeprot/* || true
dos2unix hingeprot/runHingeProt.pl 2>/dev/null || true

echo "Done setup."


(Reading database ... 117639 files and directories currently installed.)
Preparing to unpack gcc-3.4-base_3.4.6-6ubuntu3_amd64.deb ...
Unpacking gcc-3.4-base (3.4.6-6ubuntu3) over (3.4.6-6ubuntu3) ...
Preparing to unpack libg2c0_3.4.6-6ubuntu3_amd64.deb ...
Unpacking libg2c0 (1:3.4.6-6ubuntu3) over (1:3.4.6-6ubuntu3) ...
Setting up gcc-3.4-base (3.4.6-6ubuntu3) ...
Setting up libg2c0 (1:3.4.6-6ubuntu3) ...
Processing triggers for libc-bin (2.35-0ubuntu3.11) ...
Done setup.


Cloning into 'hingeprot-colab-ui'...
W: Skipping acquire of configured file 'main/source/Sources' as repository 'https://r2u.stat.illinois.edu/ubuntu jammy InRelease' does not seem to provide it (sources.list entry misspelt?)
/sbin/ldconfig.real: /usr/local/lib/libtbbbind_2_5.so.3 is not a symbolic link

/sbin/ldconfig.real: /usr/local/lib/libtbbmalloc_proxy.so.2 is not a symbolic link

/sbin/ldconfig.real: /usr/local/lib/libtbbmalloc.so.2 is not a symbolic link

/sbin/ldconfig.real: /usr/local/lib/libumf.so.1 is not a symbolic link

/sbin/ldconfig.real: /usr/local/lib/libur_adapter_opencl.so.0 is not a symbolic link

/sbin/ldconfig.real: /usr/local/lib/libhwloc.so.15 is not a symbolic link

/sbin/ldconfig.real: /usr/local/lib/libtbbbind.so.3 is not a symbolic link

/sbin/ldconfig.real: /usr/local/lib/libtbbbind_2_0.so.3 is not a symbolic link

/sbin/ldconfig.real: /usr/local/lib/libtbb.so.12 is not a symbolic link

/sbin/ldconfig.real: /usr/local/lib/libtcm_debug.so.1 is not a symboli

In [ ]:
# @title HingeProt Colab UI (FULL runNMA + writes cutoffs/rescale + chain extraction + zip)
from google.colab import output, files
output.enable_custom_widget_manager()

import os, re, shutil, subprocess, zipfile, datetime, base64, glob, textwrap
import requests
import ipywidgets as W
from IPython.display import display, clear_output, HTML

# ============================================================
# 0) CONFIG: your repo URL (edit if needed)
# ============================================================
GIT_REPO_URL = "https://github.com/enesemretas/hingeprot-colab-ui.git"
REPO_ROOT_DEFAULT = "/content/hingeprot-colab-ui"

RUNS_ROOT = "/content/hingeprot_runs"
os.makedirs(RUNS_ROOT, exist_ok=True)

# ============================================================
# 1) OS helpers
# ============================================================
def sh(cmd: str, check=True):
    return subprocess.run(["bash","-lc", cmd], check=check, capture_output=True, text=True)

def ensure_deps():
    # Keep it minimal + helpful for 32-bit binaries
    sh("apt-get update -qq", check=False)
    sh("apt-get install -y -qq perl zip unzip dos2unix libc6-i386 lib32stdc++6", check=False)

def find_repo_root():
    # If the expected path exists, use it
    if os.path.isfile(os.path.join(REPO_ROOT_DEFAULT, "hingeprot", "runNMA.pl")):
        return REPO_ROOT_DEFAULT

    # Otherwise, search /content for hingeprot/runNMA.pl
    hits = glob.glob("/content/**/hingeprot/runNMA.pl", recursive=True)
    if hits:
        return os.path.abspath(os.path.join(os.path.dirname(hits[0]), ".."))

    return None

def clone_repo_if_needed():
    root = find_repo_root()
    if root:
        return root

    # Clone into the expected folder name so paths match
    if os.path.isdir(REPO_ROOT_DEFAULT):
        shutil.rmtree(REPO_ROOT_DEFAULT, ignore_errors=True)

    sh(f"cd /content && git clone {GIT_REPO_URL} hingeprot-colab-ui", check=True)
    root = find_repo_root()
    if not root:
        raise RuntimeError("Repo cloned but hingeprot/runNMA.pl still not found. Check your repo contents/branch.")
    return root

def chmod_exec(folder):
    # Make everything in hingeprot executable (safe)
    sh(f"chmod +x {folder}/* || true", check=False)

def dos2unix_inplace(p: str):
    # Avoid dependency; normalize line endings
    try:
        with open(p, "rb") as f:
            b = f.read()
        b2 = b.replace(b"\r\n", b"\n")
        if b2 != b:
            with open(p, "wb") as f:
                f.write(b2)
    except Exception:
        pass

# ============================================================
# 2) Patch runNMA.pl so it uses local $home (FindBin) not /home/appserv/...
# ============================================================
def patch_runNMA_home(runNMA_path: str):
    txt = open(runNMA_path, "r", encoding="utf-8", errors="ignore").read()
    if "FindBin" in txt and "my $home" in txt and "$FindBin::Bin" in txt:
        return  # already patched

    # Replace: my $home="...";  -> use FindBin; my $home="$FindBin::Bin";
    lines = txt.splitlines(True)
    out = []
    inserted_findbin = False
    patched_home = False

    for line in lines:
        if (not inserted_findbin) and re.match(r"^\s*use\s+File::Copy\s*;\s*$", line):
            out.append(line)
            out.append("use FindBin;\n")
            inserted_findbin = True
            continue

        m = re.match(r'^\s*my\s+\$home\s*=\s*".*?"\s*;\s*$', line)
        if m and not patched_home:
            out.append('my $home="$FindBin::Bin";\n')
            patched_home = True
        else:
            out.append(line)

    if not inserted_findbin:
        # insert after "use strict;" if File::Copy line not found
        patched2 = []
        inserted = False
        for line in out:
            patched2.append(line)
            if (not inserted) and re.match(r"^\s*use\s+strict\s*;\s*$", line):
                patched2.append("use FindBin;\n")
                inserted = True
        out = patched2

    if not patched_home:
        raise RuntimeError("Could not find 'my $home=\"...\";' in runNMA.pl to patch. Please share your runNMA.pl header.")

    bak = runNMA_path + ".bak"
    if not os.path.exists(bak):
        shutil.copy2(runNMA_path, bak)
    with open(runNMA_path, "w", encoding="utf-8") as f:
        f.writelines(out)

# ============================================================
# 3) Core runner utilities
# ============================================================
def _fetch_pdb(pdb_code: str, out_path: str):
    code = pdb_code.strip().upper()
    if not re.fullmatch(r"[0-9A-Z]{4}", code):
        raise ValueError("PDB code must be 4 characters (e.g., 4CLN).")
    url = f"https://files.rcsb.org/download/{code}.pdb"
    r = requests.get(url, timeout=45)
    r.raise_for_status()
    if len(r.text) < 200:
        raise RuntimeError("Downloaded PDB looks too small; please check the code.")
    with open(out_path, "w", encoding="utf-8") as f:
        f.write(r.text)
    dos2unix_inplace(out_path)

def _detect_chains(pdb_path: str):
    chains = []
    seen = set()
    with open(pdb_path, "r", encoding="utf-8", errors="ignore") as f:
        for line in f:
            if line.startswith(("ATOM  ", "HETATM")) and len(line) > 21:
                ch = line[21].strip()
                if ch and ch not in seen:
                    seen.add(ch)
                    chains.append(ch)
    return chains

def _safe_rm(path: str):
    if os.path.isdir(path):
        shutil.rmtree(path, ignore_errors=True)
    elif os.path.exists(path):
        try: os.remove(path)
        except: pass

def _zip_folder(folder_path: str, zip_path: str):
    with zipfile.ZipFile(zip_path, "w", zipfile.ZIP_DEFLATED) as z:
        for root, _, files_ in os.walk(folder_path):
            for fn in files_:
                full = os.path.join(root, fn)
                rel = os.path.relpath(full, folder_path)
                z.write(full, rel)

def _read_small_text(path: str, max_chars=8000):
    if not path or (not os.path.isfile(path)):
        return ""
    with open(path, "r", encoding="utf-8", errors="ignore") as f:
        return f.read(max_chars)

def _write_params(hingeprot_dir: str, gnm: float, anm: float, rescale: float):
    # write into hingeprot folder (as requested)
    with open(os.path.join(hingeprot_dir, "gnmcutoff"), "w") as f: f.write(str(float(gnm)))
    with open(os.path.join(hingeprot_dir, "anmcutoff"), "w") as f: f.write(str(float(anm)))
    with open(os.path.join(hingeprot_dir, "rescale"),  "w") as f: f.write(str(float(rescale)))

def _extract_chains_into_pdb(hingeprot_dir: str, run_dir: str, pdb_name: str, chain_string: str):
    """
    Like server: getChain.Linux CHAINS pdbfile > pdbfileCHAINS ; rename back to pdbfile
    """
    getchain = os.path.join(hingeprot_dir, "getChain.Linux")
    if not os.path.isfile(getchain):
        raise RuntimeError("getChain.Linux not found in hingeprot folder.")
    src = os.path.join(run_dir, pdb_name)
    tmp = os.path.join(run_dir, f"{pdb_name}.{chain_string}.tmp")

    with open(tmp, "w", encoding="utf-8") as fout:
        proc = subprocess.run([getchain, chain_string, pdb_name], cwd=run_dir, stdout=fout, stderr=subprocess.PIPE, text=True)

    if proc.returncode != 0:
        raise RuntimeError(f"getChain.Linux failed (rc={proc.returncode}):\n{proc.stderr}")

    # sanity check output size
    if (not os.path.isfile(tmp)) or os.path.getsize(tmp) < 200:
        raise RuntimeError("Chain-extracted PDB is empty/too small. Check chain IDs.")
    shutil.move(tmp, src)
    dos2unix_inplace(src)

def _run_runNMA(hingeprot_dir: str, run_dir: str, pdb_name: str, log_print):
    runNMA = os.path.join(hingeprot_dir, "runNMA.pl")
    if not os.path.isfile(runNMA):
        raise RuntimeError("runNMA.pl not found in hingeprot folder.")

    # runNMA copies params from $home (hingeprot_dir) into run_dir, then runs binaries from $home
    cmd = ["perl", runNMA, pdb_name]
    log_print(f"Running: {' '.join(cmd)}")
    proc = subprocess.run(cmd, cwd=run_dir, capture_output=True, text=True)

    if proc.stdout.strip():
        log_print(proc.stdout.rstrip())
    if proc.stderr.strip():
        log_print(proc.stderr.rstrip())

    return proc.returncode

def _check_required_files(hingeprot_dir: str):
    req = [
        "runNMA.pl","read","gnmc","anm2","useblz","anm3","extract","coor2pdb",
        "processHinges","splitter","getChain.Linux",
        "gnmcutoff","anmcutoff","rescale"
    ]
    missing = [x for x in req if not os.path.exists(os.path.join(hingeprot_dir, x))]
    return missing

# ============================================================
# 4) UI helper: List/Custom float rows (GNM/ANM/Rescale)
# ============================================================
def list_or_custom_float(label: str, options, default_value: float,
                         minv: float, maxv: float, step: float = 0.1,
                         label_width="130px", toggle_width="180px", value_width="240px"):
    opts = [float(x) for x in options]
    default_value = float(default_value)
    if default_value not in opts:
        opts = sorted(set(opts + [default_value]))
    else:
        opts = sorted(set(opts))

    lbl = W.Label(label, layout=W.Layout(width=label_width))
    toggle = W.ToggleButtons(
        options=[("List","list"),("Custom","custom")],
        value="list",
        layout=W.Layout(width=toggle_width),
        style={"button_width":"80px"}
    )
    dd = W.Dropdown(options=opts, value=default_value, layout=W.Layout(width=value_width))
    bx = W.BoundedFloatText(value=default_value, min=minv, max=maxv, step=step, layout=W.Layout(width=value_width))
    valbox = W.Box([dd], layout=W.Layout(align_items="center"))

    def on_toggle(ch):
        valbox.children = [dd] if ch["new"]=="list" else [bx]
    toggle.observe(on_toggle, names="value")

    def getv():
        return float(dd.value) if toggle.value=="list" else float(bx.value)

    return W.HBox([lbl, toggle, valbox], layout=W.Layout(align_items="center", gap="12px")), getv, toggle, dd, bx

# ============================================================
# 5) Setup repo + deps now
# ============================================================
ensure_deps()
REPO_ROOT = clone_repo_if_needed()
HINGEPROT_DIR = os.path.join(REPO_ROOT, "hingeprot")
chmod_exec(HINGEPROT_DIR)

# Patch runNMA.pl to use local $home
patch_runNMA_home(os.path.join(HINGEPROT_DIR, "runNMA.pl"))

missing = _check_required_files(HINGEPROT_DIR)

# ============================================================
# 6) UI
# ============================================================
css = W.HTML("""
<style>
.hp-card {border:1px solid #e5e7eb; border-radius:14px; padding:14px 16px; margin:10px 0; background:#fff;}
.hp-title {display:flex; align-items:center; gap:10px; margin-bottom:10px;}
.hp-title b {font-size:20px;}
.hp-small {font-size:12px; color:#6b7280; margin-top:6px;}
.hp-warn {color:#b45309; font-weight:600;}
</style>
""")

header = W.HTML(f"""
<div class="hp-card">
  <div class="hp-title">
    <span style="display:inline-block;width:14px;height:14px;background:#ef4444;border-radius:999px;"></span>
    <b>HingeProt / runNMA (Google Colab UI)</b>
  </div>
  <div class="hp-small">Repo root: <code>{REPO_ROOT}</code></div>
  <div class="hp-small">HingeProt dir: <code>{HINGEPROT_DIR}</code></div>
  {"<div class='hp-warn'>Missing files: " + ", ".join(missing) + "</div>" if missing else ""}
</div>
""")

# Input mode
input_mode = W.RadioButtons(
    options=[("Enter PDB code","code"),("Upload PDB file","upload")],
    value="code",
    description="Input:",
    style={"description_width":"60px"},
    layout=W.Layout(width="420px")
)

pdb_code = W.Text(
    value="",
    description="PDB code:",
    placeholder="e.g., 4cln",
    style={"description_width":"80px"},
    layout=W.Layout(width="420px")
)

btn_choose_file = W.Button(description="Choose file", icon="upload", layout=W.Layout(width="200px"))
file_lbl = W.Label("No file chosen")

code_box = W.HBox([pdb_code], layout=W.Layout(align_items="center"))
upload_box = W.HBox([btn_choose_file, file_lbl], layout=W.Layout(align_items="center", gap="10px"))

btn_load = W.Button(
    description="Load / Detect Chains",
    button_style="info",
    icon="search",
    layout=W.Layout(width="320px")  # avoid truncation
)

chains_select = W.SelectMultiple(
    options=[],
    description="Select Chains:",
    rows=8,
    style={"description_width":"120px"},
    layout=W.Layout(width="420px")
)
all_structure = W.Checkbox(value=False, description="All structure (run all detected chains)")

# Cutoffs
gnm_row, get_gnm, gnm_toggle, gnm_dd, gnm_bx = list_or_custom_float(
    "GNM cutoff (Å):", [7,8,9,10,11,12,13,20], 10.0, 1.0, 100.0, 0.1
)
anm_row, get_anm, anm_toggle, anm_dd, anm_bx = list_or_custom_float(
    "ANM cutoff (Å):", [10,13,15,18,20,23,36], 18.0, 1.0, 100.0, 0.1
)
# Rescale/perturb (server feature)
res_row, get_res, res_toggle, res_dd, res_bx = list_or_custom_float(
    "Rescale (perturb):", [1.0, 0.5, 2.0, 3.0], 1.0, 0.01, 20.0, 0.01
)

email = W.Text(
    value="",
    description="E-mail:",
    placeholder="optional (UI parity)",
    style={"description_width":"80px"},
    layout=W.Layout(width="420px")
)

progress = W.IntProgress(value=0, min=0, max=1, description="Progress:", bar_style="")
btn_run  = W.Button(description="Submit", button_style="success", icon="play", layout=W.Layout(width="180px"))
btn_clear= W.Button(description="Clear",  button_style="warning", icon="trash", layout=W.Layout(width="180px"))

log_out = W.Output()
preview = W.Textarea(value="", description="Preview:", layout=W.Layout(width="100%", height="220px"))

state = {
    "run_dir": None,
    "pdb_path": None,
    "upload_name": None,
    "upload_bytes": None,
    "detected_chains": []
}

def log(msg: str):
    with log_out:
        print(msg)

def sync_visibility(*_):
    if input_mode.value == "code":
        code_box.layout.display = ""
        upload_box.layout.display = "none"
    else:
        code_box.layout.display = "none"
        upload_box.layout.display = ""
sync_visibility()
input_mode.observe(lambda ch: sync_visibility(), names="value")

# --- One-click JS uploader (no extra bottom button) ---
def js_upload_callback(payload):
    try:
        name = payload.get("name", "upload.pdb")
        data_b64 = payload.get("data_b64", "")
        if not data_b64:
            log("Upload callback received empty data.")
            return
        data = base64.b64decode(data_b64.encode("utf-8"))
        state["upload_name"] = name
        state["upload_bytes"] = data
        file_lbl.value = name
        log(f"Uploaded file received: {name} ({len(data)} bytes)")
    except Exception as e:
        log(f"Upload callback error: {e}")

try:
    output.register_callback("hingeprot_uploader", js_upload_callback)
except Exception:
    # safe if re-run cell
    pass

def on_choose_file(_):
    js = r"""
    (async () => {
      const input = document.createElement('input');
      input.type = 'file';
      input.accept = '.pdb,.ent';
      input.style.display = 'none';
      document.body.appendChild(input);

      input.onchange = async () => {
        const file = input.files && input.files[0];
        document.body.removeChild(input);
        if (!file) return;

        const reader = new FileReader();
        reader.onload = async () => {
          const b64 = (reader.result || "").split(",")[1] || "";
          await google.colab.kernel.invokeFunction(
            "hingeprot_uploader",
            [{name: file.name, data_b64: b64}],
            {}
          );
        };
        reader.readAsDataURL(file);
      };

      input.click();
    })();
    """
    output.eval_js(js)

btn_choose_file.on_click(on_choose_file)

def on_load(_):
    preview.value = ""
    with log_out:
        clear_output()

    if missing:
        log("ERROR: Required hingeprot files are missing in your repo. Check GitHub upload (screenshot shows they should exist).")
        log("Missing: " + ", ".join(missing))
        return

    try:
        ts = datetime.datetime.now().strftime("%Y%m%d_%H%M%S")
        run_dir = os.path.join(RUNS_ROOT, f"run_{ts}")
        os.makedirs(run_dir, exist_ok=True)
        state["run_dir"] = run_dir

        pdb_path = os.path.join(run_dir, "input.pdb")

        if input_mode.value == "upload":
            if state["upload_bytes"] is None:
                raise ValueError("Please click 'Choose file' and upload a PDB first.")
            with open(pdb_path, "wb") as f:
                f.write(state["upload_bytes"])
            dos2unix_inplace(pdb_path)
            log(f"Saved uploaded PDB: {state['upload_name']}")
        else:
            code = pdb_code.value.strip()
            if not code:
                raise ValueError("Please enter a PDB code (e.g., 4cln).")
            log(f"Downloading PDB {code} ...")
            _fetch_pdb(code, pdb_path)

        state["pdb_path"] = pdb_path
        log(f"Input PDB saved: {pdb_path} (size={os.path.getsize(pdb_path)} bytes)")

        chs = _detect_chains(pdb_path)
        if not chs:
            raise RuntimeError("No chains detected in the PDB.")
        state["detected_chains"] = chs
        chains_select.options = chs
        chains_select.value = tuple(chs[:1])

        log(f"Detected chains: {chs}")
        log("Now choose chain(s) (or All structure), then Submit.")
    except Exception as e:
        log(f"ERROR: {e}")

def on_submit(_):
    preview.value = ""
    with log_out:
        clear_output()

    if missing:
        log("ERROR: Required hingeprot files are missing. Fix repo upload first.")
        log("Missing: " + ", ".join(missing))
        return

    try:
        if not state["run_dir"] or not state["pdb_path"] or not os.path.exists(state["pdb_path"]):
            raise RuntimeError("Click 'Load / Detect Chains' first.")

        chs = state["detected_chains"]
        if not chs:
            raise RuntimeError("No detected chains. Load again.")

        if all_structure.value:
            chosen = chs
        else:
            chosen = list(chains_select.value)
        if not chosen:
            raise RuntimeError("No chain selected.")

        chain_string = "".join(chosen)  # server-style "ABC"
        gnm = float(get_gnm())
        anm = float(get_anm())
        res = float(get_res())

        # Write params into hingeprot folder (as requested)
        _write_params(HINGEPROT_DIR, gnm, anm, res)

        log(f"Parameters written in hingeprot folder:")
        log(f"  gnmcutoff = {gnm}")
        log(f"  anmcutoff = {anm}")
        log(f"  rescale   = {res}")
        if email.value.strip():
            log(f"E-mail (UI only): {email.value.strip()}")

        log("-" * 70)
        log(f"Run folder: {state['run_dir']}")
        log(f"Selected chains (combined): {chain_string}")
        log("-" * 70)

        progress.max = 3
        progress.value = 0
        progress.bar_style = "info"

        run_dir = state["run_dir"]
        pdb_name = "input.pdb"
        pdb_path = os.path.join(run_dir, pdb_name)

        # 1) Extract selected chains into the run PDB (like the server)
        _extract_chains_into_pdb(HINGEPROT_DIR, run_dir, pdb_name, chain_string)
        progress.value = 1
        log("Chain extraction done (input.pdb now contains only selected chains).")

        # 2) Run full NMA pipeline
        rc = _run_runNMA(HINGEPROT_DIR, run_dir, pdb_name, log)
        progress.value = 2
        if rc != 0:
            log(f"WARNING: runNMA.pl returned non-zero rc={rc} (some outputs may still exist).")

        # 3) Zip ALL outputs from run folder
        zip_path = os.path.join(run_dir, f"hingeprot_runNMA_{chain_string}.zip")
        _zip_folder(run_dir, zip_path)
        progress.value = 3
        progress.bar_style = "success"
        log("-" * 70)
        log(f"ZIP created: {zip_path}")

        # Preview: show any hinge/loops file we can find
        candidates = []
        # most informative
        candidates += glob.glob(os.path.join(run_dir, "*.new.hinges"))
        candidates += glob.glob(os.path.join(run_dir, "*.hinge"))
        candidates += glob.glob(os.path.join(run_dir, "*.loops"))
        candidates += glob.glob(os.path.join(run_dir, "*.slowmodes"))
        candidates = [c for c in candidates if os.path.isfile(c)]

        if candidates:
            candidates.sort(key=lambda p: os.path.getsize(p), reverse=True)
            top = candidates[0]
            preview.value = f"--- Preview file: {os.path.basename(top)} ---\n" + _read_small_text(top, max_chars=8000)
        else:
            preview.value = "(No *.new.hinges / *.hinge / *.loops / *.slowmodes found to preview. Check ZIP.)"

        files.download(zip_path)

    except Exception as e:
        progress.bar_style = "danger"
        log(f"ERROR: {e}")

def on_clear(_):
    pdb_code.value = ""
    input_mode.value = "code"

    state["upload_name"] = None
    state["upload_bytes"] = None
    file_lbl.value = "No file chosen"

    chains_select.options = []
    chains_select.value = ()
    all_structure.value = False
    state["detected_chains"] = []

    # reset rows
    gnm_toggle.value = "list"; gnm_dd.value = 10.0; gnm_bx.value = 10.0
    anm_toggle.value = "list"; anm_dd.value = 18.0; anm_bx.value = 18.0
    res_toggle.value = "list"; res_dd.value = 1.0;  res_bx.value = 1.0

    email.value = ""
    progress.value = 0
    progress.max = 1
    progress.bar_style = ""

    state["run_dir"] = None
    state["pdb_path"] = None

    with log_out:
        clear_output()
    preview.value = ""

btn_load.on_click(on_load)
btn_run.on_click(on_submit)
btn_clear.on_click(on_clear)

form = W.VBox([
    W.HTML('<div class="hp-card">'),
    W.HTML("<b>Input</b>"),
    input_mode,
    code_box,
    upload_box,
    btn_load,
    W.HTML("<hr>"),
    W.HBox([all_structure, chains_select]),
    W.VBox([gnm_row, anm_row, res_row], layout=W.Layout(gap="8px")),
    email,
    progress,
    W.HBox([btn_run, btn_clear]),
    W.HTML("</div>")
])

outbox = W.VBox([
    W.HTML('<div class="hp-card"><b>Run Log</b></div>'),
    log_out,
    W.HTML('<div class="hp-card"><b>Preview</b><div class="hp-small">Shows the beginning of a representative output file</div></div>'),
    preview
])

display(css, header, form, outbox)


HTML(value='\n<style>\n.hp-card {border:1px solid #e5e7eb; border-radius:14px; padding:14px 16px; margin:10px …

HTML(value='\n<div class="hp-card">\n  <div class="hp-title">\n    <span style="display:inline-block;width:14p…

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>